In [ ]:
#!pip install qiskit
#!pip install qiskit[visualization]

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer, AerSimulator
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2

### Initial parameters

In [ ]:
n = 10
n_cube = pow(n,3)
norm = np.sqrt(n_cube)
nr_bits = int(np.floor(np.log2(n_cube)) + 1)
dims_value = int(pow(2,nr_bits))

qb_per_process = int(np.ceil(np.log2(n_cube)))
total_qubits = n * qb_per_process

### Quantum Factory

In [ ]:
class QuantumFactory:
    def __init__(self) -> None:
        self.coin = None
        self.leader = None
        self.generate_coin_circuit()
        #self.generate_leader_circuit()
        self.generate_superposition_leader()

    def generate_coin_circuit(self):
        qc = QuantumCircuit(n)
        qc.h(0)
        qc.cx(0,range(1,n))
        qc.measure_all()    
        self.coin = qc
    
    def tensor_n_qudits(self, qudit, nr_times):
        partial_copy = qudit.copy()
        tensor_vec = qudit.tensor(partial_copy)
        for i in range(1,nr_times-1):
                tensor_vec = tensor_vec.tensor(partial_copy)
        return tensor_vec
    
    def generate_leader_circuit(self):
        qudit = Statevector.from_int(1,dims=dims_value)
        final_ket = (1 / norm) * self.tensor_n_qudits(qudit,n)
        #final_ket = (1 / norm) * qudit

        for i in range(2,n_cube+1):
            qudit = Statevector.from_int(i,dims=dims_value)
            final_ket += (1 / norm) * self.tensor_n_qudits(qudit,n)
            #final_ket += (1 / norm) * qudit

        circuit_nr_bits = n * nr_bits
        #circuit_nr_bits = nr_bits
        another_qc = QuantumCircuit(circuit_nr_bits)
        another_qc.initialize(final_ket.data)
        another_qc.measure_all()
        self.leader = another_qc

    def generate_superposition_leader(self):
        qc = QuantumCircuit(total_qubits)
        qc.h(range(0, qb_per_process))

        for j in range(1,n):
            for i in range(0, qb_per_process):
                qc.cx(i, i + j * qb_per_process)
        qc.measure_all()
        self.leader = qc

    def get_coin_circuit(self) -> QuantumCircuit:
         return self.coin.copy()
    
    def get_leader_circuit(self) -> QuantumCircuit:
         return self.leader.copy()

In [ ]:
class Circuit():
    def __init__(self, system) -> None:
        self.system = system
        self.result = None
        self.memory = None
    def measure_circuit(self):
        backend = Aer.get_backend('aer_simulator')
        self.system.save_statevector()
        self.result = backend.run(self.system, memory=True, shots=1).result()
        self.memory = self.result.get_memory()[0]
    def measure_circuit_sampler(self):
        aer_sim = AerSimulator(method="stabilizer")
        #aer_sim = AerSimulator()
        pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
        isa_qc = pm.run(self.system)
        sampler = SamplerV2(backend=aer_sim)
        
        result = sampler.run([isa_qc], shots=1).result()
        data_pub = result[0].data
        counts = data_pub.meas.get_counts()
        self.memory = list(counts.keys())[0]

### Send messages between processes
sender → sender_id 

receivers → set of processes meant to receive the message

circuit → message from sender to receiver(s)

In [ ]:
class Message:
    def __init__(self, sender, receivers, system) -> None:
        self.sender = sender
        self.receivers = receivers
        self.circuit = Circuit(system)
    def __str__(self):
        return f"sender: {self.sender} | receivers: {self.receivers} | memory: {self.circuit.memory}"
    

### Global variables

In [ ]:
quantum_factory = QuantumFactory()
coin_msgs = []
leader_msgs = []

### Acquire coin state (for each process)

In [ ]:
def send_coin(process_id):
    coin_qc = quantum_factory.get_coin_circuit()
    new_msg = Message(process_id, list(range(0,n)), coin_qc)
    coin_msgs.append(new_msg)

### Acquire Leader state (for each process)

In [ ]:
def send_leader(process_id):
    leader_qc = quantum_factory.get_leader_circuit()
    new_msg = Message(process_id, list(range(0,n)), leader_qc)
    leader_msgs.append(new_msg)

### QuantumCoinFlip

In [ ]:
def get_highest_leader():
    leader_measurements = {}
    for i in range(len(leader_msgs)):
            process_id = leader_msgs[i].sender
            leader_outcome = leader_msgs[i].circuit.memory[:nr_bits]  # with my exact approach

            existing_ids = leader_measurements.get(leader_outcome)
            if existing_ids is not None:
                existing_ids.append(process_id)
            else:
                existing_ids = [process_id]

            leader_measurements.update({leader_outcome:existing_ids})
    print(leader_measurements)
    highest_leader_outcome = max(leader_measurements, key=lambda x:x[0])    
    leader_processes_ids = leader_measurements.get(highest_leader_outcome)
    leader_processes_ids.sort()
    print("after sort: ", leader_processes_ids)
    print("highest_leader: ", leader_processes_ids[0])
    return leader_processes_ids[0]

In [ ]:
for i in range(n):
    send_coin(i)
    send_leader(i)

for i in range(len(leader_msgs)):
    leader_msgs[i].circuit.measure_circuit_sampler()
    print("leader: ", int(leader_msgs[i].circuit.memory, 2), leader_msgs[i].circuit.memory)

leader_process_id  = get_highest_leader()

leader_coin = None

for i in range(len(coin_msgs)):
    if coin_msgs[i].sender == leader_process_id:
        coin_msgs[i].circuit.measure_circuit_sampler()
        leader_coin = coin_msgs[i].circuit.memory
#print("highest_leader: ", leader_process_id)
print("leader_coin: ", leader_coin)